In [1]:
import os 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,datasets

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", n

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # or any {'0','1','2'}

In [3]:
(x,y),_ = datasets.mnist.load_data()

In [4]:
x.shape

(60000, 28, 28)

In [6]:
y.shape

(60000,)

In [7]:
def prepare_mnist_features_and_labels(x,y):
    x = tf.cast(x,tf.float32) / 255.0
    y = tf.cast(y,tf.int64)
    return x,y

In [24]:
def mnist_dataset():
    (x,y),_ = datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x,y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

In [25]:
temp = mnist_dataset()

In [27]:
features,label = iter(temp).next()
print("example features:", features[0])
print("example label:", label[0])

example features: tf.Tensor(
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.  

In [ ]:
model = keras.Sequential([
    layers.Reshape(target_shape=(28*28,),input_shape=(28,28))
    layers.Dense(100,activation='relu')
    layers.Dense(100,activation='relu')
    layers.Dense(10)
])